In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

stop_Words = text.ENGLISH_STOP_WORDS


In [2]:
df = pd.read_csv("./data/parsed/training.csv", dtype=str)
df.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,"Fri, 15 Jun 2001 11:37:53 -0700 (PDT)",david.allan@enron.com,"david.allan@enron.com, e..carter@enron.com, el...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"Allan, David </O=ENRON/OU=NA/CN=RECIPIENTS/CN=...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,"Mon, 6 Aug 2001 10:47:06 -0700 (PDT)",stacey.wales@enron.com,"stacey.wales@enron.com, e..carter@enron.com, p...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"Wales, Stacey </O=ENRON/OU=NA/CN=RECIPIENTS/CN...",...,NaN,NaN,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [47]:
import nltk, re

In [7]:
words = set(nltk.corpus.words.words())

#df["Message-Contents"].apply(len)
#df["Message-Contents"][16]
gross_email = df[df["Message-Contents"].str.contains("----")]


In [53]:
import re

'''
Email filter class that will hold all of our regexes that we can apply to ALL emails. It is helpful 
to have it as an objectbecause we can just add the regexes as instance variables and then iterate 
over all of them using <object>.__dict__.items()
'''
class Email_filter:
     def __init__(self):
        ##regexes#
        self.quoted = re.compile(r"^>(\s)*", re.MULTILINE)
        self.original_or_forward = re.compile(r"(^(\s)*-*(\s)*Original Message(\s)*-.*$)|(^____.*$)", re.MULTILINE)
        self.metadata = re.compile(r"(^From:.*$)|(^Sent:.*$)|(^To:.*$)|(^Subject:.*$)|(^Cc:.*$)|(^Date:.*$)|(^Encoding:.*$)", re.MULTILINE)
        self.ole_object = re.compile(r"(<<.*?>>)+", re.DOTALL|re.MULTILINE)
        self.smtp_header = re.compile(r"^Message-id:.*?X-Mozilla-Status.*?$", re.DOTALL|re.MULTILINE)
        self.received = re.compile(r"^Received:(.*?)\([A-Z]{3}\)", re.DOTALL|re.MULTILINE)

'''
Method to actually filter the emails - creates an email filter object and loops through all of the 
regexes it contains, running each of them on an email. If the email is a reply, it will first 
cut out all of the replies.
'''
def filter_email(s):
    if type(s) != str:
        return("")
    e = Email_filter()
    ret = s
    for variable_name, regex in e.__dict__.items():  
        ret = re.sub(regex, "", ret)
    
    return ret

'''
Method to cut out all of the replies of emails
'''
def filter_reply(s):
    replies = re.compile(r"(^(\s)*-*(\s)*Original Message.*)", re.MULTILINE|re.DOTALL)
    ret = re.sub(replies, "", s)
    return ret


'''
metadata parsing
'''
from datetime import datetime
import re
def parse_date(d):
    d = d.replace(',', '')
    redundancy_filter = d.split(' (')
    string = redundancy_filter[0]
    datetime_object = datetime.strptime(string, '%a %d %b %Y %X %z')
    return datetime_object


def x_strip(s):
    if s != "":
        array = re.split(r'</O.*?\>', s)
        array = [x.strip() for x in array]
        return array
    else:
        return([])

def to_from_strip(s):
    if s != "":
        return s.split(',')
    else:
        return([])
    
def parse_metadata(df):
    df['Date'] = df['Date'].apply(parse_date)
    df['To'] = df['To'].apply(to_from_strip)
    df['From'] = df['From'].apply(to_from_strip)
    df['X-To'] = df['X-To'].apply(x_strip)
    df['X-From'] = df['X-From'].apply(x_strip)
    
    return(df)
    
def full_filter_email(df):
    df = df.replace(np.nan, '', regex=True)
    df.loc[df["Subject"].str.lower().str.startswith('re')]["Message-Contents"].apply(filter_reply)
    df["Message-Contents"] = df["Message-Contents"].apply(filter_email)
    df = parse_metadata(df)
    return(df)

In [54]:
gross_email_filtered = pd.DataFrame()
gross_email_filtered = full_filter_email(df)

In [6]:
gross_email_filtered.head()

,Unnamed: 0,Unnamed: 0.1,Date,From,To,Subject,Mime-Version,Content-Type,Content-Transfer-Encoding,X-From,...,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message-ID,Message-Contents,ID,Label,Scenario
0,0,0,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,401
1,1,1,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,402
2,2,2,2001-06-15 11:37:53-07:00,[david.allan@enron.com],"[david.allan@enron.com, e..carter@enron.com, ...",pdated: weekly warrior meeting,1.0,text/plain; charset=us-ascii,7bit,"[Allan, David, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<22329315.1075853164077.JavaMail.evans@thyme>,"Team room - EB2942, NC",3.438093.NRVQMDFEDQVCPPCQH1LAKFSPGODI4KJHA,-1,403
3,3,3,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,401
4,4,4,2001-08-06 10:47:06-07:00,[stacey.wales@enron.com],"[stacey.wales@enron.com, e..carter@enron.com,...",Canceled: Pulp Origination Strategy,1.0,text/plain; charset=us-ascii,7bit,"[Wales, Stacey, ]",...,,,\MCAUSHOL (Non-Privileged)\Calendar,causholli-m,MCAUSHOL (Non-Privileged).pst,<19614702.1075853164028.JavaMail.evans@thyme>,"Dear Team,\n\n\nI would like to brainstorm ide...",3.438120.GTSTTLNTZ2LVIWQFALPJFBLRPP2UC0G4A,-1,402


In [55]:
gross_email_filtered.to_pickle('pickled_data_test.pickle')

In [13]:
print(gross_email["Subject"][50846])
print(filter_email(gross_email["Message-Contents"][50846], is_reply=False))

RE: Transfer of Dana Davis


TypeError: filter_email() got an unexpected keyword argument 'is_reply'

In [16]:
print(gross_email["Message-Contents"][50846])
print(gross_email_filtered["Message-Contents"][50846])

Please check with Maria Lebeau who has been coordinating hiring in the GCP group.

 -----Original Message-----
From: 	McLoughlin, Hector  
Sent:	Friday, November 02, 2001 2:20 PM
To:	Boals, Adrial; Davis, Dana; Scribner, James; Carrizales, Blanca
Cc:	Campos, Karen E.; Ormston, Kevin; Perkins, Ramona
Subject:	Transfer of Dana Davis

I'm seeking some clarification about the possible transfer of Dana Davis into James Scribner's cost center.    It is my understanding that she was to transfer effective 11/1/01?    Was the position by any chance posted?  Do we need to create a position or is it a transfer into an existing position?
Thanks for your help, 
h
Please check with Maria Lebeau who has been coordinating hiring in the GCP group.







I'm seeking some clarification about the possible transfer of Dana Davis into James Scribner's cost center.    It is my understanding that she was to transfer effective 11/1/01?    Was the position by any chance posted?  Do we need to create a position

## TF-IDF

In [4]:

emails = pd.read_pickle("data/parsed/pickles/pickled_data_test.pickle")

scenario_1 = df[emails["Scenario"] == '401']
scenario_2 = df[emails["Scenario"] == '402']
scenario_3 = df[emails["Scenario"] == '403']

def build_TFIDF_Matrix(df):
    vectorizer = TfidfVectorizer(stop_words = stop_Words, min_df = .0005)
    vectorized = vectorizer.fit_transform(df["Message-Contents"])

    dumb_numbers = [s for s in vectorizer.get_feature_names()
    if (("0" in s) or ("1" in s) or ("2" in s) or ("3" in s) or ("4" in s)
    or ("5" in s) or ("6" in s) or ("7" in s) or ("8" in s) or ("9" in s) or ("_" in s))]

    stop_words = text.ENGLISH_STOP_WORDS.union(dumb_numbers)

    vectorizer = TfidfVectorizer(stop_words = stop_words, min_df = .0005)
    vectorized = vectorizer.fit_transform(df["Message-Contents"])
    return vectorizer, vectorized

In [5]:
scenario_1_tfidf_matrix = build_TFIDF_Matrix(scenario_1)
feature_names = scenario_1_tfidf_matrix[0].get_feature_names()
print(scenario_1_tfidf_matrix[1].toarray().shape)
scenario_1_tfidf = scenario_1_tfidf_matrix[1].toarray()
np.save('scenario_1_tfidf.npy', scenario_1_tfidf)
print(type(feature_names))
#scenario_1_data_frame = pd.DataFrame(data = scenario_1_tfidf, index = scenario_1["ID"], columns = scenario_1_tfidf_matrix[0].get_feature_names())
# scenario_1_data_frame.to_pickle('data/parsed/pickles/pickled_scenario_1_tfidf.pickle')



(116399, 12135)
<class 'list'>


In [60]:
scenario_1_loaded = np.load('scenario_1_tfidf.npy')
print(scenario_1_loaded.shape)

(116399, 12135)


In [59]:
print(scenario_1_loaded)

NameError: name 'scenario_1_loaded' is not defined

In [42]:
scenario_2_tfidf_matrix = build_TFIDF_Matrix(scenario_2)
print(scenario_2_tfidf_matrix[1].toarray().shape)

scenario_2_tfidf = scenario_2_tfidf_matrix[1].toarray()
np.save('scenario_2_tfidf.npy', scenario_2_tfidf)
# scenario_2_data_frame = pd.DataFrame(data = scenario_2_tfidf, index = scenario_1["ID"], columns = scenario_2_tfidf_matrix[0].get_feature_names())

(116399, 12135)


In [43]:
scenario_3_tfidf_matrix = build_TFIDF_Matrix(scenario_3)
print(scenario_3_tfidf_matrix[1].toarray().shape)

scenario_3_tfidf = scenario_3_tfidf_matrix[1].toarray()
np.save('scenario_3_tfidf.npy', scenario_3_tfidf)
# scenario_3_data_frame = pd.DataFrame(data = scenario_3_tfidf, index = scenario_1["ID"], columns = scenario_3_tfidf_matrix[0].get_feature_names())

(116399, 12135)


AttributeError: get_feature_names not found

In [11]:
print(type(vectorizer))
print(type(vectorized))

NameError: name 'vectorizer' is not defined

## Testing code, small sample set

In [6]:
emails_short = emails[:20000]
vectorized_short = vectorizer.fit_transform(emails_short["Message-Contents"])
vectorized_short = pd.DataFrame(vectorized_short.toarray())
vectorized_short_T = vectorized_short.T
print(vectorized_short.shape)
feature_names = vectorizer.get_feature_names()
# print(len(feature_names))


(20000, 18104)


In [ ]:
feature_names.to_pickle('pickled_names.pickle')

In [11]:
print(type(vectorized_short))
print(vectorizer.get_feature_names()[:1000])

<class 'scipy.sparse.csr.csr_matrix'>
['000033', '000mt', '000s', '003366', '01730', '02081', '025866', '0337', '0354', '04075', '05500', '0551', '0748', '0f91f2936ed2981d3c03c3ee', '100080', '10013', '1004052048', '100mil', '10_year', '110101', '110180', '11112001', '112801_01_01', '112801_01_02', '112801_04_01', '112801_131_01', '112801_131_02', '112801_131_03', '113001', '11579', '116832', '121301', '1299', '13191369', '13214116', '13214487', '1365', '13897', '13905', '13906', '13914', '13917', '13920', '13924', '13938', '13943', '13947', '13950', '13955', '13959', '13966', '13968', '13969', '13972', '13988', '14364', '14367', '14370', '14382', '14383', '14388', '14389', '14390', '14392', '14393', '14399', '14400', '14401', '14404', '14410', '14414', '14420', '14421', '14471', '14472', '14481', '14483', '14485', '14492', '14494', '14495', '14503', '14515', '14519', '14525', '14526', '14542', '14543', '14544', '14566', '14567', '14568', '14570', '14573', '14582', '14586', '14587', '1

## LSA
We take the tf-idf matrix and run TruncatedSVD on it with whatever number of dimensions we want to reduce it to. The issue is I don't believe I can pull the categories. I can pull the highest ranking words from each category, but there is no storage of the category iteself. Currently a WIP on pulling each category, but for right now I'm doing the dot product so I can map term to term.

In [3]:
vectorizer = TfidfVectorizer(stop_words = stop_Words, min_df = .0005)
lsa = np.load('lsa_output.npy')

In [5]:

# # Implemented without stemming to better work with current tf-idf code and the nasty documents

svd = TruncatedSVD(n_components = 100, algorithm='randomized')

# lsa = svd.fit_transform(scenario_1_tfidf)

def get_closest(term, vectorizer ,model):
 
    index = vectorizer.vocabulary_[term]

    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return vectorizer.get_feature_names()[out]
    else:
        return vectorizer.get_feature_names()[(out+1)]
    
def get_closest_document(index, vectorizer, model):
    model = np.dot(model, model.T)
    search_space = np.concatenate( (model[index][:index] , model[index][(index+1):]) )  
    out = np.argmax(search_space)
    if out<index:
        return emails_short["Message-Contents"][out]
    else:
        return emails_short["Message-Contents"][out+1]
#     if out<index:
#         return vectorizer.get_feature_names()[out]
#     else:
#         return vectorizer.get_feature_names()[(out+1)]
    
def get_k_closest(k,term,vectorizer,model):
    index = vectorizer.vocabulary_[term]
    print("index: ", index)

    # This is one approach where we take the dot product
    # This results in a mapping of terms to terms based on similarity to each other
    model = np.dot(model,model.T)
    print(model.shape)

    closest_terms = {}
    print(len(model))
    for i in range(len(model)):
        closest_terms[vectorizer.get_feature_names()[i]] = model[index][i]
    #print(closest_terms)
    sorted_list = sorted(closest_terms , key = lambda l : closest_terms[l])
    return sorted_list[::-1][0:k]
            

    


In [17]:
print(get_closest("stock", vectorizer, lsa))
print(get_k_closest(5, "stock", vectorizer, lsa))

IndexError: index 44374 is out of bounds for axis 0 with size 20000

## get_closest_document example

Outputs the email that is the most similar to the one given (by index). Interesting example here that show another email about meeting with someone named Monika. Super close match for a small sample size.

In [ ]:
print(get_closest_document(510, vectorizer, lsa))
print("--------------------------------------------------------------")
print(emails_short["Message-Contents"][510])

## Pulling the categories

Here we have the words that form the category for category \#50. "Monika" is the defining person of category \#50, as are "wrap", "market", and "report".

You can take absolute values 

In [10]:
svd_components = np.load('svd-components.npy')

def full_category(svd_components, feature_names, index):
    temp = abs(svd_components[index]).argsort()
    svd_sorted = abs(svd_components[index])[temp]
    print('\n'.join(["%0.7f*%s" % (coef, feat) for coef, feat in zip(svd_sorted, np.array(feature_names)[temp])]))

full_category(svd_components, feature_names, 40)

0.0000000*hgovenar
0.0000000*mcnichols
0.0000000*annoying
0.0000000*howell
0.0000001*explosion
0.0000003*medical
0.0000004*carmen
0.0000005*umbrella
0.0000008*isn
0.0000008*cromwell
0.0000009*dws
0.0000010*dasr
0.0000010*erroneously
0.0000011*bb
0.0000011*bengals
0.0000013*cheatsheets
0.0000017*gerry
0.0000017*fundies
0.0000018*wade
0.0000018*plummer
0.0000018*separated
0.0000019*mansoor
0.0000020*possession
0.0000022*keyshawn
0.0000022*rented
0.0000022*redmond
0.0000023*stoler
0.0000023*marketplace
0.0000023*panthers
0.0000023*groin
0.0000024*guns
0.0000027*wm
0.0000028*sutherland
0.0000030*asian
0.0000031*revolutionary
0.0000032*packaging
0.0000033*initiated
0.0000033*shops
0.0000035*toyota
0.0000035*sells
0.0000036*maturity
0.0000037*bedroom
0.0000038*ridiculous
0.0000038*worker
0.0000039*bottle
0.0000039*terance
0.0000039*drivers
0.0000039*netcom
0.0000041*tonite
0.0000042*commissions
0.0000043*arms
0.0000043*wyoming
0.0000044*undoubtedly
0.0000044*remove
0.0000045*defect
0.0000046

In [31]:
import pickle

with open('feature_names_short.pickle', 'wb') as f:
    pickle.dump(feature_names_short, f)

<class 'list'>
